# Performance graphs in the sepsis treatment simulator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle


def load_obj(name,dir_name):
    with open(dir_name + 'obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
plt.rcParams.update({'font.size': 12})
plt.rcParams.update({'font.size': 12})

In [ ]:
plt.rcParams.update({'font.size': 12})


In [ ]:
def make_graphs():
    
    repeats = 5
    sample_step = 10
    iters1 = 0
    iters2 = 0
    iters3 = 0
    
    dlistE = []
    dlistBB1 = []
    dlistBB2 = []
    
    accurE = []
    accurBB1 = []
    accurBB2 = []
    # plot accuracy:
    for i in range(repeats):
        try:
            accurE.append(np.load("./Linear/Ellipsoid/obj/accuracy_" + str(i) + ".npy"))
            
        except:
            pass
        
    for i in range(repeats):
        try:
            accurBB1.append(np.load("./Linear/Blackbox_loss1/obj/accuracy_" + str(i) + ".npy"))
            
        except:
            pass
        
    for i in range(repeats):
        try:
            accurBB2.append(np.load("./Linear/Blackbox_loss2/obj/accuracy_" + str(i) + ".npy"))
            
        except:
            pass
        
        
    for i in range(repeats):
        try:
            dlistE.append(load_obj("values"+str(i),'Linear/Ellipsoid/'))
            if iters1 == 0:
                iters1 = len(dlistE[0][0,"expert_values"])
        except:
            pass
    dlistBB1 = []
    for i in range(repeats):
        try:
            dlistBB1.append(load_obj("values"+str(i),'Linear/Blackbox_loss1/'))
            if iters2 == 0:
                iters2 = len(dlistBB1[0][0,"expert_values"])
        except:
            pass
    dlistBB2 = []
    for i in range(repeats):
        try:
            dlistBB2.append(load_obj("values"+str(i),'Linear/Blackbox_loss2/'))
            if iters3 == 0:
                iters3 = len(dlistBB2[0][0,"expert_values"])
        except:
            pass
        

    # Build data Ellipsoid:
    expert_valuesE =    np.zeros((repeats,iters1))
    agent_valuesE =     np.zeros((repeats,iters1))
    cumm_regretE =      np.zeros((repeats,iters1))
    W_L2_distE =        np.zeros((repeats,iters1))
    W_Li_distE =        np.zeros((repeats,iters1))
    W_angular_distE =   np.zeros((repeats,iters1))
    contexts_seenE =    np.zeros((repeats,iters1))
    contexts_seen_minE = np.inf
    for trainset in range(repeats):
        for d in dlistE:
            if (trainset,"expert_values") in d:
                expert_valuesE[trainset] =    d[trainset,"expert_values"]
                agent_valuesE[trainset] =     d[trainset,"agent_values"]
                cumm_regretE[trainset] =      d[trainset,"cumm_regret"]
                contexts_seenE[trainset] =    d[trainset,"contexts_seen"]
                contexts_seen_minE = min(contexts_seen_minE, contexts_seenE[trainset][-1])

    print("all ellipsoid runs had at least ",int(contexts_seen_minE)," contexts")
    contexts_seen_minE -= contexts_seen_minE%sample_step
    contexts_seen_minE = int(contexts_seen_minE)+sample_step
    test_expert_valueE = np.zeros((repeats,int(contexts_seen_minE/sample_step)))
    test_agent_valueE = np.zeros((repeats,int(contexts_seen_minE/sample_step)))
    test_W_L2_distE   = np.zeros((repeats,int(contexts_seen_minE/sample_step)))
    test_W_Li_distE   = np.zeros((repeats,int(contexts_seen_minE/sample_step)))
    test_W_angular_distE = np.zeros((repeats,int(contexts_seen_minE/sample_step)))
    TESTS = False
    
    
    for trainset in range(repeats):
        for d in dlistE:
                if (trainset,"test_value",0) in d:
                    TESTS = True
                    for contexts in range(0,contexts_seen_minE,sample_step):
                        test_expert_valueE[trainset][int(contexts/sample_step)] = d["test_value"]
                        test_agent_valueE[trainset][int(contexts/sample_step)] = d[trainset,"test_value",contexts]

    expert_valuesE = expert_valuesE[~np.all(expert_valuesE == 0, axis=1)]
    agent_valuesE = agent_valuesE[~np.all(agent_valuesE == 0, axis=1)]
    cumm_regretE = cumm_regretE[~np.all(cumm_regretE == 0, axis=1)]
    W_Li_distE = W_Li_distE[~np.all(W_Li_distE == 0, axis=1)]
    W_L2_distE = W_L2_distE[~np.all(W_L2_distE == 0, axis=1)]
    W_angular_distE = W_angular_distE[~np.all(W_angular_distE == 0, axis=1)]
    contexts_seenE = contexts_seenE[~np.all(contexts_seenE == 0, axis=1)]
    test_expert_valueE = test_expert_valueE[~np.all(test_expert_valueE == 0, axis=1)]
    test_agent_valueE = test_agent_valueE[~np.all(test_agent_valueE == 0, axis=1)]
    test_W_L2_distE = test_W_L2_distE[~np.all(test_W_L2_distE == 0, axis=1)]
    test_W_Li_distE = test_W_Li_distE[~np.all(test_W_Li_distE == 0, axis=1)]
    test_W_angular_distE = test_W_angular_distE[~np.all(test_W_angular_distE == 0, axis=1)]
    print("Number of runs used in graph:",len(expert_valuesE))
    print("Timesteps in each run:",len(expert_valuesE[0]))
    
    
    expert_values_stdE =     expert_valuesE.std(axis=0)
    agent_values_stdE =      agent_valuesE.std(axis=0)
    expert_values_meanE =    expert_valuesE.mean(axis=0)
    agent_values_meanE =     agent_valuesE.mean(axis=0)
    cumm_regret_meanE =      cumm_regretE.mean(axis=0)
    cumm_regret_stdE =       cumm_regretE.std(axis=0)
    W_L2_dist_stdE =           W_L2_distE.std(axis=0)
    W_Li_dist_stdE =           W_Li_distE.std(axis=0)
    W_L2_dist_meanE =           W_L2_distE.mean(axis=0)
    W_Li_dist_meanE =           W_Li_distE.mean(axis=0)
    W_angular_dist_stdE =   W_angular_distE.std(axis=0)
    W_angular_dist_meanE =   W_angular_distE.mean(axis=0)
    contexts_seen_stdE =     contexts_seenE.std(axis=0)
    contexts_seen_meanE =    contexts_seenE.mean(axis=0)
    

    # Build data Blackbox 1st loss:
    expert_valuesBB1 =    np.zeros((repeats,iters2))
    agent_valuesBB1 =     np.zeros((repeats,iters2))
    cumm_regretBB1 =      np.zeros((repeats,iters2))
    W_L2_distBB1 =        np.zeros((repeats,iters2))
    W_Li_distBB1 =        np.zeros((repeats,iters2))
    W_angular_distBB1 =   np.zeros((repeats,iters2))
    contexts_seenBB1 =    np.zeros((repeats,iters2))
    contexts_seen_minBB1 = np.inf
    adv = 0
    for trainset in range(repeats):
        for d in dlistBB1:
            if (trainset,"expert_values") in d:
                expert_valuesBB1[adv] =    d[trainset,"expert_values"]
                agent_valuesBB1[adv] =     d[trainset,"agent_values"]
                cumm_regretBB1[adv] =      d[trainset,"cumm_regret"]
                contexts_seenBB1[adv] =    d[trainset,"contexts_seen"]
                contexts_seen_minBB1 = min(contexts_seen_minBB1, contexts_seenBB1[adv][-1])
                adv += 1
                
    print("all blackbox with 1st loss runs had at least ",int(contexts_seen_minBB1)," contexts")
    contexts_seen_minBB1 -= contexts_seen_minBB1%sample_step
    contexts_seen_minBB1 = int(contexts_seen_minBB1)+sample_step
    test_expert_valueBB1 = np.zeros((repeats,int(contexts_seen_minBB1/sample_step)))
    test_agent_valueBB1 = np.zeros((repeats,int(contexts_seen_minBB1/sample_step)))
    test_W_L2_distBB1   = np.zeros((repeats,int(contexts_seen_minBB1/sample_step)))
    test_W_Li_distBB1   = np.zeros((repeats,int(contexts_seen_minBB1/sample_step)))
    test_W_angular_distBB1 = np.zeros((repeats,int(contexts_seen_minBB1/sample_step)))
    
    TESTS = False
    adv = 0
    for trainset in range(repeats):
        for d in dlistBB1:
                if (trainset,"test_value",0) in d:
                    TESTS = True
                    for contexts in range(0,contexts_seen_minBB1,sample_step):
                        test_expert_valueBB1[adv][int(contexts/sample_step)] = d["test_value"]
                        test_agent_valueBB1[adv][int(contexts/sample_step)] = d[trainset,"test_value",contexts]
                    adv += 1

    expert_valuesBB1 = expert_valuesBB1[~np.all(expert_valuesBB1 == 0, axis=1)]
    agent_valuesBB1 = agent_valuesBB1[~np.all(agent_valuesBB1 == 0, axis=1)]
    cumm_regretBB1 = cumm_regretBB1[~np.all(cumm_regretBB1 == 0, axis=1)]
    contexts_seenBB1 = contexts_seenBB1[~np.all(contexts_seenBB1 == 0, axis=1)]
    test_expert_valueBB1 = test_expert_valueBB1[~np.all(test_expert_valueBB1 == 0, axis=1)]
    test_agent_valueBB1 = test_agent_valueBB1[~np.all(test_agent_valueBB1 == 0, axis=1)]
    print("Number of runs used in graph:",len(expert_valuesBB1))
    print("Timesteps in each run:",len(expert_valuesBB1[0]))
    
    
    expert_values_stdBB1 =     expert_valuesBB1.std(axis=0)
    agent_values_stdBB1 =      agent_valuesBB1.std(axis=0)
    expert_values_meanBB1 =    expert_valuesBB1.mean(axis=0)
    agent_values_meanBB1 =     agent_valuesBB1.mean(axis=0)
    cumm_regret_meanBB1 =      cumm_regretBB1.mean(axis=0)
    cumm_regret_stdBB1 =       cumm_regretBB1.std(axis=0)
    contexts_seen_stdBB1 =     contexts_seenBB1.std(axis=0)
    contexts_seen_meanBB1 =    contexts_seenBB1.mean(axis=0)
    
    
    # Build data Blackbox 2nd loss:
    expert_valuesBB2 =    np.zeros((repeats,iters2))
    agent_valuesBB2 =     np.zeros((repeats,iters2))
    cumm_regretBB2 =      np.zeros((repeats,iters2))
    W_L2_distBB2 =        np.zeros((repeats,iters2))
    W_Li_distBB2 =        np.zeros((repeats,iters2))
    W_angular_distBB2 =   np.zeros((repeats,iters2))
    contexts_seenBB2 =    np.zeros((repeats,iters2))
    contexts_seen_minBB2 = np.inf
    adv = 0
    for trainset in range(repeats):
        for d in dlistBB2:
            if (trainset,"expert_values") in d:
                expert_valuesBB2[adv] =    d[trainset,"expert_values"]
                agent_valuesBB2[adv] =     d[trainset,"agent_values"]
                cumm_regretBB2[adv] =      d[trainset,"cumm_regret"]
                contexts_seenBB2[adv] =    d[trainset,"contexts_seen"]
                contexts_seen_minBB2 = min(contexts_seen_minBB2, contexts_seenBB2[adv][-1])
                adv += 1
                
    print("all blackbox with 2nd loss runs had at least ",int(contexts_seen_minBB2)," contexts")
    contexts_seen_minBB2 -= contexts_seen_minBB2%sample_step
    contexts_seen_minBB2 = int(contexts_seen_minBB2)+sample_step
    test_expert_valueBB2 = np.zeros((repeats,int(contexts_seen_minBB2/sample_step)))
    test_agent_valueBB2 = np.zeros((repeats,int(contexts_seen_minBB2/sample_step)))
    test_W_L2_distBB2   = np.zeros((repeats,int(contexts_seen_minBB2/sample_step)))
    test_W_Li_distBB2   = np.zeros((repeats,int(contexts_seen_minBB2/sample_step)))
    test_W_angular_distBB2 = np.zeros((repeats,int(contexts_seen_minBB2/sample_step)))
    
    TESTS = False
    adv = 0
    for trainset in range(repeats):
        for d in dlistBB2:
                if (trainset,"test_value",0) in d:
                    TESTS = True
                    for contexts in range(0,contexts_seen_minBB2,sample_step):
                        test_expert_valueBB2[adv][int(contexts/sample_step)] = d["test_value"]
                        test_agent_valueBB2[adv][int(contexts/sample_step)] = d[trainset,"test_value",contexts]
                    adv += 1

    expert_valuesBB2 = expert_valuesBB2[~np.all(expert_valuesBB2 == 0, axis=1)]
    agent_valuesBB2 = agent_valuesBB2[~np.all(agent_valuesBB2 == 0, axis=1)]
    cumm_regretBB2 = cumm_regretBB2[~np.all(cumm_regretBB2 == 0, axis=1)]
    contexts_seenBB2 = contexts_seenBB2[~np.all(contexts_seenBB2 == 0, axis=1)]
    test_expert_valueBB2 = test_expert_valueBB2[~np.all(test_expert_valueBB2 == 0, axis=1)]
    test_agent_valueBB2 = test_agent_valueBB2[~np.all(test_agent_valueBB2 == 0, axis=1)]
    print("Number of runs used in graph:",len(expert_valuesBB2))
    print("Timesteps in each run:",len(expert_valuesBB2[0]))
    
    
    expert_values_stdBB2 =     expert_valuesBB2.std(axis=0)
    agent_values_stdBB2 =      agent_valuesBB2.std(axis=0)
    expert_values_meanBB2 =    expert_valuesBB2.mean(axis=0)
    agent_values_meanBB2 =     agent_valuesBB2.mean(axis=0)
    cumm_regret_meanBB2 =      cumm_regretBB2.mean(axis=0)
    cumm_regret_stdBB2 =       cumm_regretBB2.std(axis=0)
    contexts_seen_stdBB2 =     contexts_seenBB2.std(axis=0)
    contexts_seen_meanBB2 =    contexts_seenBB2.mean(axis=0)
    

    if TESTS:
        # Value graph (contexts)
        fig = plt.figure(figsize=(4,3))
        p = fig.add_subplot(111)
        
        test_expert_value_meanE = test_expert_valueE.mean(axis=0)
        test_expert_value_stdE = test_expert_valueE.std(axis=0)
        test_agent_value_meanE = test_agent_valueE.mean(axis=0)
        test_agent_value_stdE = test_agent_valueE.std(axis=0)

        p.plot(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE, label="Expert", color='black')
        p.plot(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE, label="Ellipsoid", color='g')

        p.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE - test_expert_value_stdE, test_expert_value_meanE + test_expert_value_stdE, color='black', alpha=0.1)
        p.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE - test_agent_value_stdE, test_agent_value_meanE + test_agent_value_stdE, color='g', alpha=0.1)

        test_expert_value_meanBB1 = test_expert_valueBB1.mean(axis=0)
        test_expert_value_stdBB1 = test_expert_valueBB1.std(axis=0)
        test_agent_value_meanBB1 = test_agent_valueBB1.mean(axis=0)
        test_agent_value_stdBB1 = test_agent_valueBB1.std(axis=0)

#         p.plot(list(range(0,contexts_seen_minB,5)), test_expert_value_meanB, label="Expert", color='b')
        p.plot(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1, label="ES, loss (1)", color='r')

#         p.fill_between(list(range(0,contexts_seen_minB,5)), test_expert_value_meanB - test_expert_value_stdB, test_expert_value_meanB + test_expert_value_stdB, color='b', alpha=0.1)
        p.fill_between(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1 - test_agent_value_stdBB1, test_agent_value_meanBB1 + test_agent_value_stdBB1, color='r', alpha=0.1)

        test_expert_value_meanBB2 = test_expert_valueBB2.mean(axis=0)
        test_expert_value_stdBB2 = test_expert_valueBB2.std(axis=0)
        test_agent_value_meanBB2 = test_agent_valueBB2.mean(axis=0)
        test_agent_value_stdBB2 = test_agent_valueBB2.std(axis=0)

#         p.plot(list(range(0,contexts_seen_minB,5)), test_expert_value_meanB, label="Expert", color='b')
        p.plot(list(range(0,contexts_seen_minBB2,sample_step)), test_agent_value_meanBB2, label="ES, loss (2)", color='b')

#         p.fill_between(list(range(0,contexts_seen_minB,5)), test_expert_value_meanB - test_expert_value_stdB, test_expert_value_meanB + test_expert_value_stdB, color='b', alpha=0.1)
        p.fill_between(list(range(0,contexts_seen_minBB2,sample_step)), test_agent_value_meanBB2 - test_agent_value_stdBB2, test_agent_value_meanBB2 + test_agent_value_stdBB2, color='b', alpha=0.1)
    
    
    
        p.set_xlabel("Demonstrations")
        p.set_ylabel("Value")
        p.tick_params('y', colors='k')
        p.set_ylim(0.014,0.021)
#         plt.title("Avg. value on test set")
        p.legend(loc=8)
        plt.show()
        
        minlen_accurE = accurE[0].size
        for i in range(len(accurE)):
            if accurE[i].size < minlen_accurE:
                minlen_accurE = accurE[i].size
        
        accuracyE = np.zeros([len(accurE),minlen_accurE])
        for i in range(len(accurE)):
            accuracyE[i,:] = accurE[i][:minlen_accurE]
    
        minlen_accurBB1 = accurBB1[0].size
        for i in range(len(accurBB1)):
            if accurBB1[i].size < minlen_accurBB1:
                minlen_accurBB1 = accurBB1[i].size
                
        accuracyBB1 = np.zeros([len(accurBB1),minlen_accurBB1])
        for i in range(len(accurBB1)):
            accuracyBB1[i,:] = accurBB1[i][:minlen_accurBB1]
            
        minlen_accurBB2 = accurBB2[0].size
        for i in range(len(accurBB2)):
            if accurBB2[i].size < minlen_accurBB2:
                minlen_accurBB2 = accurBB2[i].size
                
        accuracyBB2 = np.zeros([len(accurBB2),minlen_accurBB2])
        for i in range(len(accurBB2)):
            accuracyBB2[i,:] = accurBB2[i][:minlen_accurBB2]
            
        accuracy_meanE = 1 - accuracyE.mean(axis=0)
        accuracy_stdE = accuracyE.std(axis=0)
        accuracy_meanBB1 = 1 - accuracyBB1.mean(axis=0)
        accuracy_stdBB1 = accuracyBB1.std(axis=0)
        accuracy_meanBB2 = 1 - accuracyBB2.mean(axis=0)
        accuracy_stdBB2 = accuracyBB2.std(axis=0)
   
        fig5 = plt.figure(figsize=(4,3))
        p5 = fig5.add_subplot(111)
        p5.set_xlim(8,200)
        p = p5.twinx()
        lin0 = p.plot(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE, label="Expert", color='black')
        p5.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE - test_expert_value_stdE, test_expert_value_meanE + test_expert_value_stdE, color='black', alpha=0.1)
        lin1 = p5.plot(list(range(0,minlen_accurE*sample_step,sample_step)), 100*accuracy_meanE, label="Ellipsoid", color='#145a32')
        p5.fill_between(list(range(0,minlen_accurE*sample_step,sample_step)), 100*accuracy_meanE - 100*accuracy_stdE, 100*accuracy_meanE + 100*accuracy_stdE, color='#145a32', alpha=0.1)
        
        lin2 = p5.plot(list(range(0,minlen_accurBB1*sample_step,sample_step)), 100*accuracy_meanBB1, label="ES, loss (1)", color='#a93226')
        p5.fill_between(list(range(0,minlen_accurBB1*sample_step,sample_step)), 100*accuracy_meanBB1 - 100*accuracy_stdBB1, 100*accuracy_meanBB1 + 100*accuracy_stdBB1, color='#a93226', alpha=0.1)

        lin3 = p5.plot(list(range(0,minlen_accurBB2*sample_step,sample_step)), 100*accuracy_meanBB2, label="ES, loss (2)", color='#1f618d')
        p5.fill_between(list(range(0,minlen_accurBB2*sample_step,sample_step)), 100*accuracy_meanBB2 - 100*accuracy_stdBB2, 100*accuracy_meanBB2 + 100*accuracy_stdBB2, color='#1f618d', alpha=0.1)
        
        lin4 = p.plot(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE, label="Ellipsoid", color='g')
        p.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE - test_agent_value_stdE, test_agent_value_meanE + test_agent_value_stdE, color='g', alpha=0.1)
        
        lin5 = p.plot(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1, label="ES, loss (1)", color='r')
        p.fill_between(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1 - test_agent_value_stdBB1, test_agent_value_meanBB1 + test_agent_value_stdBB1, color='r', alpha=0.1)
        
        lin6 = p.plot(list(range(0,contexts_seen_minBB2,sample_step)), test_agent_value_meanBB2, label="ES, loss (2)", color='b')
        p.fill_between(list(range(0,contexts_seen_minBB2,sample_step)), test_agent_value_meanBB2 - test_agent_value_stdBB2, test_agent_value_meanBB2 + test_agent_value_stdBB2, color='b', alpha=0.1)
        


        p.set_xlabel("Demonstrations")
        p.set_ylabel("Value")
        p.set_ylim(0.012,0.0208)
#         plt.title("Avg. value on test set")
        linss = lin0+lin4+lin5+lin6+lin1+lin2+lin3
        labss =[linn.get_label() for linn in linss]
        p.legend(linss,labss,loc='center right')
        
        
        p5.set_xlabel("Demonstrations")
        p5.set_ylabel("% Miss")
        p5.set_ylim(10,100)
        p5.tick_params('y', colors='#145a32')
        p.tick_params('y', colors='g')
#         plt.title("Accuracy of learned policy(contexts seen)")
#         p5.legend(loc='center right')
        plt.show()
    
    fig = plt.figure(figsize=(4,3))
    p = fig.add_subplot(111)
    
    p.plot(list(range(iters1)), cumm_regret_meanE, label="Ellipsoid", color='g')
    p.fill_between(list(range(iters1)), cumm_regret_meanE - cumm_regret_stdE, cumm_regret_meanE + cumm_regret_stdE, color='g', alpha=0.1)
    p.plot(list(range(iters2)), cumm_regret_meanBB1, label="ES, loss (1)", color='r')
    p.fill_between(list(range(iters2)), cumm_regret_meanBB1 - cumm_regret_stdBB1, cumm_regret_meanBB1 + cumm_regret_stdBB1, color='r', alpha=0.1)
    p.plot(list(range(iters3)), cumm_regret_meanBB2, label="ES, loss (2)", color='b')
    p.fill_between(list(range(iters3)), cumm_regret_meanBB2 - cumm_regret_stdBB2, cumm_regret_meanBB2 + cumm_regret_stdBB2, color='b', alpha=0.1)
    
    p.set_xlabel("Time")
    p.set_ylabel("Regret")
    p.legend(loc='best')
#     plt.title("Cumulative regret over time")
    plt.show()


    fig = plt.figure(figsize=(4,3))
    p2 = fig.add_subplot(111)

    p2.plot(list(range(iters1)), contexts_seen_meanE, label="Ellipsoid", color='g')
    p2.fill_between(list(range(iters1)), contexts_seen_meanE - contexts_seen_stdE, contexts_seen_meanE + contexts_seen_stdE, color='g', alpha=0.1)

    
    p2.plot(list(range(iters2)), contexts_seen_meanBB1, label="ES, loss (1)", color='r')
    p2.fill_between(list(range(iters2)), contexts_seen_meanBB1 - contexts_seen_stdBB1, contexts_seen_meanBB1 + contexts_seen_stdBB1, color='r', alpha=0.1)

    
    p2.plot(list(range(iters3)), contexts_seen_meanBB2, label="ES, loss (2)", color='b')
    p2.fill_between(list(range(iters3)), contexts_seen_meanBB2 - contexts_seen_stdBB2, contexts_seen_meanBB2 + contexts_seen_stdBB2, color='b', alpha=0.1)

    p2.set_ylabel("Demonstrations")

    p2.set_xlabel("Time")

#     plt.title("Demonstrations required over time")
    p2.legend(loc='best')

    plt.show()

In [ ]:
x=make_graphs()